### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

!pip install razdel
from razdel import sentenize, tokenize
from math import log

In [2]:
answers = pd.read_excel('/content/drive/My Drive/PROGA/infosearch/hw2/answers_base.xlsx')
questions = pd.read_excel('/content/drive/My Drive/PROGA/infosearch/hw2/queries_base.xlsx')
answers.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ


In [3]:
text, train = train_test_split(questions, test_size = 0.3, random_state=67839)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(answers['Текст вопросов'])
array_answers = X.toarray()

In [4]:
array_answers.shape

(43, 1554)

In [5]:
words=vectorizer.get_feature_names()
pd_matrix_answers = pd.DataFrame(array_answers, columns=words)
pd_matrix_answers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Columns: 1554 entries, 10 to явно
dtypes: float64(1554)
memory usage: 522.2 KB


In [6]:
def vectorize(text: str):
  vec = vectorizer.transform(text)
  X = vec.toarray()[0]
  scalar = array_answers.dot(X)

  return scalar

question_array = vectorize(questions['Текст вопроса'].head(1))
question_array.shape

(43,)

In [9]:
links = list(answers['Номер связки'])

def raking(question_array):
  vec_link ={}
  for x,y in enumerate(np.nditer(question_array)):
    vec_link[float(y)] = links[x]

  zip_links = [(el, vec_link[el]) for el in sorted(vec_link.keys(), reverse=True)]

  result = 'В ранжировании выиграл документ номер: ' + str(zip_links[0][1]) + '\n'
  print(result)

  for x,y in zip_links:
    print('Коэффициент близости: ', x,'\t Номер документа: ',y)
  
  
raking(question_array)

В ранжировании выиграл документ номер: 308

Коэффициент близости:  0.186124813762977 	 Номер документа:  308
Коэффициент близости:  0.16869161558362822 	 Номер документа:  135
Коэффициент близости:  0.14332733589020885 	 Номер документа:  6
Коэффициент близости:  0.12684606813855775 	 Номер документа:  286
Коэффициент близости:  0.12635102520844524 	 Номер документа:  270
Коэффициент близости:  0.12088349071349433 	 Номер документа:  1
Коэффициент близости:  0.11765950736340816 	 Номер документа:  88
Коэффициент близости:  0.11482520646588393 	 Номер документа:  5
Коэффициент близости:  0.10432220238297922 	 Номер документа:  37
Коэффициент близости:  0.07085366550799663 	 Номер документа:  246
Коэффициент близости:  0.0652499567437455 	 Номер документа:  325
Коэффициент близости:  0.061739732054359696 	 Номер документа:  74
Коэффициент близости:  0.05591413582628507 	 Номер документа:  132
Коэффициент близости:  0.055790216712495955 	 Номер документа:  79
Коэффициент близости:  0.0513

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [30]:
list_tokens = []
def get_num_tokens(text):
    tokenized_text = tokenize(text)
    num_tokens = len(list(tokenized_text))
    tokenized_text = list(tokenized_text)
    return num_tokens,tokenized_text

for text in list(answers['Текст вопросов']):
  list_tokens.append(get_num_tokens(text)[0])

answers['кол-во токенов'] = list_tokens
answers.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика,кол-во токенов
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,146
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,119
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,22
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,44
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,11


In [32]:
k = 2.0
b = 0.75
N = len(answers['Текст вопросов'])
avgdl = answers['кол-во токенов'].mean()

def BM_25(D, Q):
  l = get_num_tokens(D)[0]
  freq = len(pd_matrix_answers[pd_matrix_answers[Q]>0])
  idf = log((N- freq + 0.5)/(freq + 0.5))
  i = 0
  for words in get_num_tokens(D)[1]:
    if Q == word:
      i+=1
  tf = i / l
  result = idf * ((tf*(k+1))/(tf+k*(1-b+b*(l/avgdl))))
  return result
  


D =  list(answers['Текст вопросов'].head(1))[0]
Q = 'covid'
BM_25(D, Q)

0.0